In [ ]:
! pip install openai

In [1]:
from langchain_community.document_transformers import (
    LongContextReorder,
)
from langchain.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter, CharacterTextSplitter
from langchain.document_loaders import TextLoader
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.storage import InMemoryStore
from langchain.retrievers.document_compressors import CohereRerank
import os
from dotenv import load_dotenv
import cohere

In [2]:
load_dotenv()
COHERE_API_KEY = os.getenv("COHERE_API_KEY")
HF_EMBEDDING = HuggingFaceEmbeddings(model_name="keepitreal/vietnamese-sbert")
vectorstore = Chroma(persist_directory="./data" ,embedding_function=HF_EMBEDDING)
retriever =vectorstore.as_retriever()

/home/tranhuy/Desktop/Project/cantho-public-services/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
question = "Muốn chuyển quyền sử dụng đất cho con thi phải làm thủ tục gì?"
docs = retriever.get_relevant_documents(question)
co = cohere.Client(api_key=COHERE_API_KEY)

docs = [reordered_doc.page_content for reordered_doc in docs]
results = co.rerank(model="rerank-multilingual-v2.0", query=question, documents=docs, top_n=3)
context = ""
for index, result in enumerate(results):
    print(result.document)

context = " ".join(results[0].document["text"].split(" ")[-4096:])


from openai import OpenAI
client = OpenAI()
inputs = f" Đây là văn bản chứa thông tin, bạn có thể dùng nếu cần:\n{context}.Hãy trả lời câu hỏi, nếu không có thông tin trong văn bản, hãy trả lời theo kiến thức của bạn về dịch vụ công ở Cần Thơ Việt Nam"

completion = client.chat.completions.create(
model="gpt-3.5-turbo",
messages=[
    {"role": "system", "content": "Bạn là một tư vấn viên cho các dịch vụ công tại Cần Thơ Việt Nam, hãy trả lời lịch sự."},
    {"role": "system", "content": inputs},
    {"role": "user", "content": question}
]
)
completion.choices[0].message.content

{'text': 'Cần chuẩn bị các hồ sơ sau đây và nộp tại Không có thông tin với thủ tục hành chính Chuyển mục đích sử dụng đất phải được phép của cơ quan nhà nước có thẩm quyền đối với tổ chức:\n1. Đơn xin phép chuyển mục đích sử dụng đất: cần 1 bản chính và 0 bản sao.. Mẫu đơn cho hồ sơ này có thể tìm thấy tại https://csdl.dichvucong.gov.vn/web/jsp/download_file.jsp?ma=3fe0dbd541e865b9\n2. Giấy chứng nhận quyền sử dụng đất hoặc Giấy chứng nhận quyền sở hữu nhà ở và quyền sử dụng đất ở hoặc Giấy chứng nhận quyền sử dụng đất, quyền sở hữu nhà ở và tài sản khác gắn liền với đất.: cần 1 bản chính và 0 bản sao.. Mẫu đơn cho hồ sơ này có thể tìm thấy tại \n3. Văn bản thẩm định điều kiện cho phép chuyển mục đích sử dụng đất đối với trường hợp chuyển mục đích sử dụng đất để thực hiện dự án đầu tư xây dựng nhà ở để bán hoặc để cho thuê hoặc để bán kết hợp cho thuê theo quy định của pháp luật về nhà ở; dự án đầu tư kinh doanh bất động sản gắn với quyền sử dụng đất theo quy định của pháp luật về kinh

'Để chuyển quyền sử dụng đất cho con, bạn cần làm thủ tục chuyển nhượng đất như sau:\n\n1. Chuẩn bị hồ sơ:\n   - Giấy chứng nhận quyền sử dụng đất hoặc Giấy chứng nhận quyền sở hữu nhà ở và quyền sử dụng đất ở hoặc Giấy chứng nhận quyền sử dụng đất, quyền sở hữu nhà ở và tài sản khác gắn liền với đất (1 bản chính và 0 bản sao).\n   - Đơn xin chuyển nhượng quyền sử dụng đất (mẫu đơn có thể tìm thấy tại https://csdl.dichvucong.gov.vn/web/jsp/download_file.jsp?ma=3fe0dbd541e865b9) (1 bản chính và 0 bản sao).\n   - Các giấy tờ tùy theo quy định của cơ quan địa phương (ví dụ: chứng minh nhân dân, sổ hộ khẩu, văn bản từ chối chuyển nhượng của các chủ thể có quyền ưu tiên mua, v.v.).\n\n2. Nộp hồ sơ:\n   - Mang hồ sơ đầy đủ đi nộp tại cơ quan nhà nước có thẩm quyền ở Cần Thơ, chẳng hạn Sở Tài nguyên và Môi trường, Phòng Tài nguyên và Môi trường phụ trách địa phương.\n   - Nếu hồ sơ hợp lệ, bạn sẽ nhận được biên nhận và số đăng ký hồ sơ.\n\n3. Xem xét và thông báo kết quả:\n   - Cơ quan nhà nư

In [4]:
with open("./answer.txt", "w") as f:
    f.write(completion.choices[0].message.content)